# Burocracias
---

In [ ]:
%pip install -r requirements.txt

In [6]:
from ucimlrepo import fetch_ucirepo
# https://archive.ics.uci.edu/dataset/10/automobile

import pandas as pd
from tailor_made_methods import check_missing_values


# Pre Processamento
---

In [7]:

# fetch dataset
automobile = fetch_ucirepo(id=10)

# data (as pandas dataframes)
y = automobile.data.features['price']

# drop price from features
X = automobile.data.features.drop('price', axis=1)

# convert to dataframe
X = pd.DataFrame(X)

# print head of X
X.head()


,highway-mpg,city-mpg,peak-rpm,horsepower,compression-ratio,stroke,bore,fuel-system,engine-size,num-of-cylinders,...,length,wheel-base,engine-location,drive-wheels,body-style,num-of-doors,aspiration,fuel-type,make,normalized-losses
0,27,21,5000.0,111.0,9.0,2.68,3.47,mpfi,130,4,...,168.8,88.6,front,rwd,convertible,2.0,std,gas,alfa-romero,NaN
1,27,21,5000.0,111.0,9.0,2.68,3.47,mpfi,130,4,...,168.8,88.6,front,rwd,convertible,2.0,std,gas,alfa-romero,NaN
2,26,19,5000.0,154.0,9.0,3.47,2.68,mpfi,152,6,...,171.2,94.5,front,rwd,hatchback,2.0,std,gas,alfa-romero,NaN
3,30,24,5500.0,102.0,10.0,3.40,3.19,mpfi,109,4,...,176.6,99.8,front,fwd,sedan,4.0,std,gas,audi,164.0
4,22,18,5500.0,115.0,8.0,3.40,3.19,mpfi,136,5,...,176.6,99.4,front,4wd,sedan,4.0,std,gas,audi,164.0


## Eliminação manual de atributos e Integração dos atributos

In [10]:
# drop price and normalized-losses from features
X = automobile.data.features.drop(['price','compression-ratio', 'peak-rpm','stroke', 'bore', 'fuel-system', 'engine-size', 'engine-type', 'curb-weight', 'num-of-doors', ], axis=1)

# print head of X
X.head()

,highway-mpg,city-mpg,horsepower,num-of-cylinders,height,width,length,wheel-base,engine-location,drive-wheels,body-style,aspiration,fuel-type,make,normalized-losses
0,27,21,111.0,4,48.8,64.1,168.8,88.6,front,rwd,convertible,std,gas,alfa-romero,NaN
1,27,21,111.0,4,48.8,64.1,168.8,88.6,front,rwd,convertible,std,gas,alfa-romero,NaN
2,26,19,154.0,6,52.4,65.5,171.2,94.5,front,rwd,hatchback,std,gas,alfa-romero,NaN
3,30,24,102.0,4,54.3,66.2,176.6,99.8,front,fwd,sedan,std,gas,audi,164.0
4,22,18,115.0,5,54.3,66.4,176.6,99.4,front,4wd,sedan,std,gas,audi,164.0


## Limpeza dos dados

In [11]:
check_missing_values(X)

[('peak-rpm', 0), ('horsepower', 0), ('normalized-losses', 3)]

In [9]:
print(X['peak-rpm'])

0      5000.0
1      5000.0
2      5000.0
3      5500.0
4      5500.0
        ...  
200    5400.0
201    5300.0
202    5500.0
203    4800.0
204    5400.0
Name: peak-rpm, Length: 205, dtype: float64


## Verificando correlação

# Modelo de regressão
---